##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 構造的なプルーニングを使用したスパースな重み

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/model_optimization/guide/pruning/pruning_with_sparsity_2_by_4.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

構造的なプルーニングでモデルの重みを特定のパターンでスパースにすると、適切なハードウェアのサポートを使用してモデルの推論時間を短縮できます。

このチュートリアルでは、次の方法を説明します。

- 特定の構造的なスパース性を備えた mnist データセットでモデルを定義およびトレーニングする
- プルーニングされたモデルを tflite 形式に変換する
- プルーニングされた重みの構造を視覚化する

モデルを最適化するためのプルーニング手法の一般的な概要については、[プルーニングの概要](https://www.tensorflow.org/model_optimization/guide/pruning)を参照してください。一般的な重みのプルーニングに関するチュートリアルについては、[Keras でのプルーニング](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)を参照してください。

## 重みの構造的なプルーニング

構造的なプルーニングは、トレーニングプロセスの開始時にモデルの重みを体系的にゼロにします。このプルーニング手法を重みの通常のブロックに適用して、ハードウェアによりサポートされる推論を高速化します。たとえば、モデル内の重みを 4 つのブロックでグループ化し、各ブロックでそれらの重みのうち 2 つをゼロにします（*{nbsp}2 x 4* のプルーニング）。この手法は、TensorFlowLite により変換されるモデルの重みテンソルの最後の次元にのみ適用されます。たとえば、TensorFlowLite の `Conv2D` レイヤーの重みの構造は `[channel_out, height, width, channel_in]` で `Dense` レイヤーの重みの構造は `[channel_out, channel_in]` です。スパースパターンは、最後の次元の重み `channel_in` に適用されます。

ランダムなスパース性と比較すると、構造的なスパース性は構造が制限されているため、一般に精度が低くなりますが、サポートされているハードウェアでの推論時間を大幅に短縮できます。

プルーニングは、他のモデル圧縮手法と共にモデルに適用して、圧縮率を向上させることができます。詳細については、[協調的最適化手法](https://blog.tensorflow.org/2021/10/Collaborative-Optimizations.html)の量子化とクラスタリングの例を参照してください。

## セットアップ

開発環境とデータを準備します。

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization
! pip install -q matplotlib

In [ ]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

## [MNIST](https://www.tensorflow.org/datasets/catalog/mnist) データセットから画像データをダウンロードして正規化する

In [ ]:
# Load MNIST dataset.
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

## 構造プルーニングのパラメータを定義する

プルーニングのパラメータを定義し、構造プルーニングの型を指定します。プルーニングのパラメータを `(2, 4)` に設定します。これらの設定は、4 要素のブロックで、少なくとも値が最も小さい 2  要素がゼロ値化されることを意味します。

`pruning_schedule` パラメータは設定する必要はありません。デフォルトでは、プルーニングマスクは最初のステップで定義され、トレーニング中に更新されません。

In [ ]:
pruning_params_2_by_4 = {
    'sparsity_m_by_n': (2, 4),
}

50％ のターゲットスパース性でランダムプルーニングのパラメータを定義します。

In [ ]:
pruning_params_sparsity_0_5 = {
    'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.5,
                                                              begin_step=0,
                                                              frequency=100)
}

モデルアーキテクチャを定義し、プルーニングするレイヤーを指定します。構造的プルーニングは、選択したモデルのレイヤーに基づいて適用されます。

以下の例では、一部のレイヤーのみをプルーニングします。2 番目の `Conv2D` レイヤーと最初の `Dense` レイヤーをプルーニングします。

最初の `Conv2D` レイヤーは構造的にプルーニングできないことに注意してください。構造的にプルーニングするには、複数の入力チャンネルが必要です。代わりに、最初の `Conv2D` レイヤーをランダムなプルーニングでプルーニングします。

In [ ]:
model = keras.Sequential([
    prune_low_magnitude(
        keras.layers.Conv2D(
            32, 5, padding='same', activation='relu',
            input_shape=(28, 28, 1),
            name="pruning_sparsity_0_5"),
        **pruning_params_sparsity_0_5),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    prune_low_magnitude(
        keras.layers.Conv2D(
            64, 5, padding='same',
            name="structural_pruning"),
        **pruning_params_2_by_4),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),
    keras.layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    keras.layers.Flatten(),
    prune_low_magnitude(
        keras.layers.Dense(
            1024, activation='relu',
            name="structural_pruning_dense"),
        **pruning_params_2_by_4),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

モデルをトレーニングして評価します。

In [ ]:
batch_size = 128
epochs = 2

model.fit(
    train_images,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    verbose=0,
    callbacks=tfmot.sparsity.keras.UpdatePruningStep(),
    validation_split=0.1)

_, pruned_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Pruned test accuracy:', pruned_model_accuracy)

プルーニングラッパーを削除し TensorFlow Lite 形式に変換するときにモデルに含まれないようにします。

In [ ]:
model = tfmot.sparsity.keras.strip_pruning(model)

## モデルを tflite 形式に変換する

In [ ]:
import tempfile

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

_, tflite_file = tempfile.mkstemp('.tflite')
print('Saved converted pruned model to:', tflite_file)
with open(tflite_file, 'wb') as f:
  f.write(tflite_model)

## 重みを視覚化して確認する

次に、2x4 のスパース性でプルーニングされた `Dense` レイヤーの重みの構造を視覚化します。tflite ファイルから重みを抽出します。

In [ ]:
# Load tflite file with the created pruned model
interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()

details = interpreter.get_tensor_details()

# Weights of the dense layer that has been pruned.
tensor_name = 'structural_pruning_dense/MatMul'
detail = [x for x in details if tensor_name in x["name"]]

# We need the first layer.
tensor_data = interpreter.tensor(detail[0]["index"])()

プルーニングされた正しいレイヤーを選択したことを確認するには、重みテンソルの形状を出力します。

In [ ]:
print(f"Shape of Dense layer is {tensor_data.shape}")

重みテンソルの小さなサブセットの構造を視覚化します。重みテンソルの構造は、`(2,4)` パターンを使用して、最後の次元でスパースです。4 要素のうち 2 要素はゼロです。視覚化をより明確にするために、ゼロ以外のすべての値を 1 に置き換えます。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# The value 24 is chosen for convenience.
width = height = 24

subset_values_to_display = tensor_data[0:height, 0:width]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(subset_values_to_display) > 0, val_ones, val_zeros)

構造を明確に確認するために分離線を描画する補助関数を定義します。

In [ ]:
def plot_separation_lines(height, width):

    block_size = [1, 4]

    # Add separation lines to the figure.
    num_hlines = int((height - 1) / block_size[0])
    num_vlines = int((width - 1) / block_size[1])
    line_y_pos = [y * block_size[0] for y in range(1, num_hlines + 1)]
    line_x_pos = [x * block_size[1] for x in range(1, num_vlines + 1)]

    for y_pos in line_y_pos:
        plt.plot([-0.5, width], [y_pos - 0.5 , y_pos - 0.5], color='w')

    for x_pos in line_x_pos:
        plt.plot([x_pos - 0.5, x_pos - 0.5], [-0.5, height], color='w')

次に、重みテンソルのサブセットを視覚化します。

In [ ]:
plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structural pruning for Dense layer")
plt.show()

`Conv2D` レイヤーの重みを視覚化します。構造的スパース性は、`Dense` レイヤーと同様に、最後のチャネルに適用されます。前述のように、2 番目の `Conv2D` レイヤーのみが構造的にプルーニングされます。

In [ ]:
# Get weights of the convolutional layer that has been pruned with 2 by 4 sparsity.
tensor_name = 'structural_pruning/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[1]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

`Dense` レイヤーの重みと同様に、カーネルの最後の次元は (2, 4) 構造を持っています。

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tf.reduce_prod(tensor_data.shape[:-1]), -1])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 1e-9, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Structurally pruned weights for Conv2D layer")
plt.show()

ランダムにプルーニングされた重みがどのようになるかを見てみましょう。それらを抽出し、重みテンソルのサブセットを表示します。

In [ ]:
# Get weights of the convolutional layer that has been pruned with random pruning.
tensor_name = 'pruning_sparsity_0_5/Conv2D'
detail = [x for x in details if tensor_name in x["name"]]
tensor_data = interpreter.tensor(detail[0]["index"])()
print(f"Shape of the weight tensor is {tensor_data.shape}")

In [ ]:
weights_to_display = tf.reshape(tensor_data, [tensor_data.shape[0],tf.reduce_prod(tensor_data.shape[1:])])
weights_to_display = weights_to_display[0:width, 0:height]

val_ones = np.ones([height, width])
val_zeros = np.zeros([height, width])
subset_values_to_display = np.where(abs(weights_to_display) > 0, val_ones, val_zeros)

plot_separation_lines(height, width)

plt.axis('off')
plt.imshow(subset_values_to_display)
plt.colorbar()
plt.title("Unstructed pruned weights for Conv2D layer")
plt.show()

TensorFlow モデル最適化ツールキットには、指定された tflite ファイルのモデル内のどのレイヤーが構造的にプルーニングされた重みを持っているかを確認するために使用できる Python スクリプト [ `check_sparsity_m_by_n.py`](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py) が含まれています。次のコマンドは、このツールを使用して、特定のモデルで 2x4 のスパース性をチェックする方法を示しています。

In [ ]:
! python3 ./tensorflow_model_optimization/python/core/sparsity/keras/tools/check_sparsity_m_by_n.py --model_tflite=pruned_model.tflite --m_by_n=2,4
